Querying data that will be used to train the neural network
---
1) querying all coins for which will build a correlated neural network model

2) keeping only the data of interest in an array:

    t-m:   p1 p2 p3 p4 ... pn
    t-m+1: p1 p2 p3 p4 ... pn
    t-m+2: p1 p2 p3 p4 ... pn 
    ...
    t:     p1 p2 p3 p4 ... pn

3) the vars array will have shape (n currencies, m points)

In [7]:
import numpy as np

from cryptocompare import get_histo

vars = []
mhd = 'hour'
candle_width = 1
m = 800
e = 'Bitfinex'
currencies = ['BTC', 'ETH', 'XRP', 'TRX', 'NEO']
n = len(currencies)
for coin in currencies:
    q = get_histo(mhd, coin, m, candle_width, e)
    data = q['Data']
    var = []
    for d0, d1 in zip(data[:-1], data[1:]):
        if d0['close'] == 0:
            p = 0
        else:
            p = d1['close']/d0['close'] - 1
        var.append(p)
    vars.append(var)
vars = np.array(vars)
print('vars.shape', vars.shape)

vars.shape (5, 800)


### Particioning blocks of data that will be used to build the neural network
---
1) determining the number of points for each block

2) building vector of inputs (xs) and outputs (ys)

3) classifying responses for each cryptocurrency using the following scheme (only 2 classes, I tried with more classes but the prediction gets much worse):

    0 -->     y < threshold
    1 -->     y >= threshold
    

In [8]:
block = 50
xs = []
ys = []
for i in range(vars.shape[1] - block):
    part = vars[:, i:i+block]
    xs.append(part[:, :-1])
    ys.append(part[:, -1])
xs = np.array(xs)
print('xs.shape', xs.shape)
def classify(y):
    threshold = 0.02
    if y < threshold:
        return 0
    else:
        return 1
    
classes = {}
for i, currency in enumerate(currencies):
    tmp = []
    for y in ys:
        tmp.append(classify(y[i]))
    classes[currency] = np.array(tmp)
    

xs.shape (750, 5, 49)


Shuffling samples for a better stochastic response
---

In [9]:
import random
rand_indices = np.arange(xs.shape[0])
random.shuffle(rand_indices)
xs = xs[rand_indices]
for k in classes:
    classes[k] = classes[k][rand_indices]
print('xs.shape', xs.shape)

xs.shape (750, 5, 49)


Training a simple Keras model
---

Reference: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [10]:
xs = xs.reshape(xs.shape[0], -1)
print('xs.shape', xs.shape)

xs.shape (750, 245)


In [11]:
from keras.models import Sequential
from keras.layers import Dense

models = {}
for k in ['BTC']:
    Y = classes[k]
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=xs.shape[1], activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    out = model.fit(xs, Y, epochs=250, batch_size=10)
    # evaluate the model
    scores = model.evaluate(xs, Y)
    print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    models[k] = model
   

Epoch 1/250
750/750 [==============================] - 0s 598us/step - loss: 0.6233 - acc: 0.9147
Epoch 2/250
750/750 [==============================] - 0s 116us/step - loss: 0.4321 - acc: 0.9147
Epoch 3/250
750/750 [==============================] - 0s 112us/step - loss: 0.3050 - acc: 0.9147
Epoch 4/250
750/750 [==============================] - 0s 123us/step - loss: 0.2804 - acc: 0.9147
Epoch 5/250
750/750 [==============================] - 0s 112us/step - loss: 0.2736 - acc: 0.9147
Epoch 6/250
750/750 [==============================] - 0s 95us/step - loss: 0.2673 - acc: 0.9147
Epoch 7/250
750/750 [==============================] - 0s 96us/step - loss: 0.2616 - acc: 0.9147
Epoch 8/250
750/750 [==============================] - 0s 112us/step - loss: 0.2552 - acc: 0.9147
Epoch 9/250
750/750 [==============================] - 0s 100us/step - loss: 0.2488 - acc: 0.9147
Epoch 10/250
750/750 [==============================] - 0s 99us/step - loss: 0.2423 - acc: 0.9160
Epoch 11/250
750/750 [

750/750 [==============================] - 0s 104us/step - loss: 0.0122 - acc: 0.9987
Epoch 85/250
750/750 [==============================] - 0s 88us/step - loss: 0.0115 - acc: 0.9987
Epoch 86/250
750/750 [==============================] - 0s 90us/step - loss: 0.0108 - acc: 1.0000
Epoch 87/250
750/750 [==============================] - 0s 95us/step - loss: 0.0100 - acc: 1.0000
Epoch 88/250
750/750 [==============================] - 0s 91us/step - loss: 0.0096 - acc: 1.0000
Epoch 89/250
750/750 [==============================] - 0s 96us/step - loss: 0.0088 - acc: 1.0000
Epoch 90/250
750/750 [==============================] - 0s 92us/step - loss: 0.0083 - acc: 1.0000
Epoch 91/250
750/750 [==============================] - 0s 87us/step - loss: 0.0078 - acc: 1.0000
Epoch 92/250
750/750 [==============================] - 0s 91us/step - loss: 0.0072 - acc: 1.0000
Epoch 93/250
750/750 [==============================] - 0s 94us/step - loss: 0.0068 - acc: 1.0000
Epoch 94/250
750/750 [==========

750/750 [==============================] - 0s 103us/step - loss: 1.6660e-04 - acc: 1.0000
Epoch 166/250
750/750 [==============================] - 0s 96us/step - loss: 1.5910e-04 - acc: 1.0000
Epoch 167/250
750/750 [==============================] - 0s 90us/step - loss: 1.5189e-04 - acc: 1.0000
Epoch 168/250
750/750 [==============================] - 0s 94us/step - loss: 1.4310e-04 - acc: 1.0000
Epoch 169/250
750/750 [==============================] - 0s 86us/step - loss: 1.4066e-04 - acc: 1.0000
Epoch 170/250
750/750 [==============================] - 0s 98us/step - loss: 1.3423e-04 - acc: 1.0000
Epoch 171/250
750/750 [==============================] - 0s 95us/step - loss: 1.2671e-04 - acc: 1.0000
Epoch 172/250
750/750 [==============================] - 0s 92us/step - loss: 1.2188e-04 - acc: 1.0000
Epoch 173/250
750/750 [==============================] - 0s 94us/step - loss: 1.1612e-04 - acc: 1.0000
Epoch 174/250
750/750 [==============================] - 0s 87us/step - loss: 1.1021e-

750/750 [==============================] - 0s 99us/step - loss: 5.5764e-06 - acc: 1.0000
Epoch 245/250
750/750 [==============================] - 0s 95us/step - loss: 5.2414e-06 - acc: 1.0000
Epoch 246/250
750/750 [==============================] - 0s 95us/step - loss: 5.1398e-06 - acc: 1.0000
Epoch 247/250
750/750 [==============================] - 0s 92us/step - loss: 4.8852e-06 - acc: 1.0000
Epoch 248/250
750/750 [==============================] - 0s 88us/step - loss: 4.6855e-06 - acc: 1.0000
Epoch 249/250
750/750 [==============================] - 0s 84us/step - loss: 4.4728e-06 - acc: 1.0000
Epoch 250/250
750/750 [==============================] - 0s 55us/step

acc: 100.00%


Testing the predictor
---

In [16]:
print('Prediction', models['BTC'].predict(xs))
print('Ref:', classes['BTC'])

Prediction [[  8.80634843e-10]
 [  9.19022909e-07]
 [  1.52701321e-14]
 [  6.01172871e-08]
 [  6.14760142e-13]
 [  3.10632164e-09]
 [  8.15196813e-07]
 [  1.37151610e-10]
 [  3.18433558e-06]
 [  9.99991298e-01]
 [  6.85322285e-11]
 [  2.30252208e-08]
 [  8.09404455e-09]
 [  1.53173108e-09]
 [  1.40414580e-09]
 [  1.46256139e-06]
 [  1.56023070e-05]
 [  1.76658905e-05]
 [  3.11557358e-08]
 [  4.87676880e-06]
 [  8.45731991e-12]
 [  1.57745028e-09]
 [  8.63068124e-07]
 [  6.25940233e-09]
 [  9.90055060e-09]
 [  1.73684359e-10]
 [  9.99999881e-01]
 [  1.45527757e-11]
 [  5.15110059e-07]
 [  3.80985954e-09]
 [  3.98724526e-11]
 [  9.99866605e-01]
 [  4.67171368e-09]
 [  1.36192238e-11]
 [  4.38369549e-11]
 [  1.09498098e-10]
 [  8.78194044e-07]
 [  6.75186200e-08]
 [  3.02389017e-12]
 [  3.25037917e-08]
 [  4.39978644e-15]
 [  2.66827359e-12]
 [  2.83181034e-08]
 [  1.53165947e-05]
 [  1.18056809e-09]
 [  9.38748741e-08]
 [  3.55041323e-07]
 [  1.57726454e-08]
 [  8.94976283e-16]
 [  9.734